In [ ]:
# ============================================================================
# COMPLETE DOUBLE ML ANALYSIS - GOOGLE COLAB
# ============================================================================

# STEP 0: DOWNLOAD DATA
!pip install -q gdown
import gdown
import os
FILE_ID = "19sq2HQipIoIcJvsID_M8iB84Ap-2lJje"
output_zip = "/content/data.zip"
url = f"https://drive.google.com/uc?id={FILE_ID}"
gdown.download(url, output_zip, quiet=False)
!unzip -q "{output_zip}" -d /content/
print("✓ Data extracted")

# STEP 1: INSTALL PACKAGES
!pip install -q python-dotenv scikit-learn linearmodels geopandas matplotlib seaborn scipy

# STEP 2: IMPORTS & CONFIGURATION
from pathlib import Path
import datetime
import json
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import box, Point
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold
from scipy import stats
import itertools
import warnings
warnings.filterwarnings('ignore')

BASE_PATH = Path('/content')
DATA_DIR = BASE_PATH / 'data'

CONFIG = {
    'study_name': 'endangered_asia_disaster_analysis_dml',
    'data_dir': DATA_DIR,
    'results_dir': BASE_PATH / 'results',
    'notebook_generated_on': datetime.datetime.utcnow().isoformat() + 'Z',
    'grid_resolution_deg': 0.5,
    'region_bbox': [60.0, -10.0, 150.0, 55.0],
    'time_unit': 'year',
    'time_range': (2000, 2024),
    'taxa': ['Aves', 'Mammalia', 'Reptilia', 'Amphibia'],
    'target_status': ['CR', 'EN', 'VU'],
    'min_occurrences': 50,
    'disaster_types': ['wildfire', 'flood', 'cyclone', 'earthquake'],
    'event_window': (-3, 5),
    'ml_model': 'random_forest',
    'n_folds': 5,
    'save_intermediate': True,
    'figure_format': 'png',
    'figure_dpi': 300,
}

for dir_path in [CONFIG['data_dir'], CONFIG['results_dir']]:
    dir_path.mkdir(parents=True, exist_ok=True)
    (dir_path / 'raw').mkdir(exist_ok=True)
    (dir_path / 'processed').mkdir(exist_ok=True)

with open(CONFIG['results_dir'] / 'config.json', 'w') as f:
    json.dump({k: str(v) if isinstance(v, Path) else v for k, v in CONFIG.items()}, f, indent=2)

print(f"✓ Config set: {CONFIG['study_name']}")

# STEP 3: DATA LOADING FUNCTIONS
def create_analysis_grid(bbox, resolution):
    min_lon, min_lat, max_lon, max_lat = bbox
    lons = np.arange(min_lon, max_lon, resolution)
    lats = np.arange(min_lat, max_lat, resolution)
    grid_polys = []
    grid_ids = []
    grid_centers = []
    for i, lon in enumerate(lons):
        for j, lat in enumerate(lats):
            poly = box(lon, lat, lon + resolution, lat + resolution)
            grid_polys.append(poly)
            grid_ids.append(f'g_{i}_{j}')
            grid_centers.append((lon + resolution/2, lat + resolution/2))
    grid_gdf = gpd.GeoDataFrame({
        'grid_id': grid_ids,
        'lon_center': [c[0] for c in grid_centers],
        'lat_center': [c[1] for c in grid_centers],
        'geometry': grid_polys
    }, crs='EPSG:4326')
    grid_gdf['area_km2'] = grid_gdf.to_crs('EPSG:3857').geometry.area / 1e6
    return grid_gdf

def map_points_to_grid(points_gdf, grid_gdf):
    pts = points_gdf.to_crs(grid_gdf.crs)
    joined = gpd.sjoin(pts, grid_gdf[['grid_id', 'geometry']], how='left', predicate='within')
    joined = joined.dropna(subset=['grid_id'])
    return joined

def load_occurrence_data(filepath, species_list, year_min, year_max, bbox):
    if not filepath.exists():
        print(f"⚠ File not found: {filepath}")
        return None
    try:
        df = pd.read_csv(filepath, sep='\t', encoding='utf-8', engine='python', quoting=3)
        if len(df.columns) < 10:
            df = pd.read_csv(filepath, sep=',', encoding='utf-8', low_memory=False)
    except:
        df = pd.read_csv(filepath, sep=',', encoding='utf-8', low_memory=False)
    print(f"Loaded {len(df):,} records")
    if 'species' in df.columns:
        df = df[df['species'].isin(species_list)]
    if 'year' in df.columns:
        df['year'] = pd.to_numeric(df['year'], errors='coerce')
        df = df[(df['year'] >= year_min) & (df['year'] <= year_max)]
    if 'decimalLatitude' in df.columns and 'decimalLongitude' in df.columns:
        df = df[(df['decimalLongitude'] >= bbox[0]) & (df['decimalLongitude'] <= bbox[2]) &
                (df['decimalLatitude'] >= bbox[1]) & (df['decimalLatitude'] <= bbox[3])]
        df = df.dropna(subset=['decimalLatitude', 'decimalLongitude'])
    print(f"After filtering: {len(df):,} records")
    return df

def load_disaster_data(filepath, disaster_types, year_min, year_max, bbox):
    if not filepath.exists():
        print(f"⚠ File not found: {filepath}")
        return None
    df = pd.read_csv(filepath, encoding='utf-8-sig', low_memory=False)
    if 'Start Year' in df.columns:
        df = df[(df['Start Year'] >= year_min) & (df['Start Year'] <= year_max)]
        df = df.rename(columns={'Start Year': 'year'})
    if 'Latitude' in df.columns and 'Longitude' in df.columns:
        df = df.dropna(subset=['Latitude', 'Longitude'])
        df = df[(df['Longitude'] >= bbox[0]) & (df['Longitude'] <= bbox[2]) &
                (df['Latitude'] >= bbox[1]) & (df['Latitude'] <= bbox[3])]
        df = df.rename(columns={'Latitude': 'latitude', 'Longitude': 'longitude'})
    print(f"Loaded {len(df):,} disaster events")
    return df

print("✓ Data loading functions defined")

# STEP 4: LOAD SPECIES LIST
species_data = {
    'taxon': ['Aves', 'Aves', 'Aves', 'Aves', 'Mammalia', 'Mammalia', 'Mammalia', 'Mammalia', 'Reptilia', 'Reptilia', 'Reptilia', 'Amphibia', 'Amphibia'],
    'scientificName': ['Lophura edwardsi', 'Arborophila davidi', 'Turdoides striata', 'Carpococcyx renauldi',
                       'Panthera tigris', 'Elephas maximus', 'Rhinoceros sondaicus', 'Pongo abelii',
                       'Crocodylus siamensis', 'Chelonia mydas', 'Cuora trifasciata',
                       'Ansonia latidisca', 'Rhacophorus catamitus'],
    'status': ['CR', 'EN', 'VU', 'EN', 'EN', 'EN', 'CR', 'CR', 'CR', 'EN', 'CR', 'EN', 'VU']
}
species_df = pd.DataFrame(species_data)
species_df.to_csv(CONFIG['data_dir'] / 'processed' / 'target_species.csv', index=False)
print(f"✓ Species list: {len(species_df)} species")

# STEP 5: CREATE SPATIAL GRID
grid_gdf = create_analysis_grid(CONFIG['region_bbox'], CONFIG['grid_resolution_deg'])
print(f"✓ Grid created: {len(grid_gdf):,} cells")
grid_path = CONFIG['data_dir'] / 'processed' / 'analysis_grid.gpkg'
grid_gdf.to_file(grid_path, driver='GPKG')

# STEP 6: LOAD OCCURRENCE DATA
gbif_file = CONFIG['data_dir'] / 'raw' / 'gbif' / '0019239-251025141854904.csv'
if gbif_file.exists():
    occurrence_df = load_occurrence_data(gbif_file, species_df['scientificName'].tolist(),
                                         CONFIG['time_range'][0], CONFIG['time_range'][1], CONFIG['region_bbox'])
    if occurrence_df is not None and len(occurrence_df) > 0:
        occ_gdf = gpd.GeoDataFrame(occurrence_df,
                                   geometry=gpd.points_from_xy(occurrence_df['decimalLongitude'],
                                                               occurrence_df['decimalLatitude']), crs='EPSG:4326')
        occ_joined = map_points_to_grid(occ_gdf, grid_gdf)
        years_list = list(range(CONFIG['time_range'][0], CONFIG['time_range'][1] + 1))
        species_list = species_df['scientificName'].tolist()
        grid_ids = grid_gdf['grid_id'].unique()
        observed = occ_joined.groupby(['grid_id', 'year', 'species']).size().reset_index(name='n_occurrences')
        observed['occupancy'] = 1
        all_combos = pd.DataFrame(list(itertools.product(grid_ids, years_list, species_list)),
                                  columns=['grid_id', 'year', 'species'])
        occurrence_panel = all_combos.merge(observed, on=['grid_id', 'year', 'species'], how='left')
        occurrence_panel['n_occurrences'] = occurrence_panel['n_occurrences'].fillna(0)
        occurrence_panel['occupancy'] = occurrence_panel['occupancy'].fillna(0)
        print(f"✓ Balanced panel: {len(occurrence_panel):,} rows, occupancy rate: {occurrence_panel['occupancy'].mean()*100:.2f}%")
        occurrence_panel.to_csv(CONFIG['data_dir'] / 'processed' / 'occurrence_panel.csv', index=False)
    else:
        occurrence_panel = None
else:
    print(f"⚠ GBIF file not found")
    occurrence_panel = None

# STEP 7: LOAD DISASTER DATA
emdat_file = CONFIG['data_dir'] / 'raw' / 'emdat_asia_2000_2024.csv'
if emdat_file.exists():
    emdat_df = load_disaster_data(emdat_file, CONFIG['disaster_types'], CONFIG['time_range'][0],
                                   CONFIG['time_range'][1], CONFIG['region_bbox'])
    if emdat_df is not None:
        disaster_gdf = gpd.GeoDataFrame(emdat_df, geometry=gpd.points_from_xy(emdat_df['longitude'],
                                                                                emdat_df['latitude']), crs='EPSG:4326')
        disaster_joined = map_points_to_grid(disaster_gdf, grid_gdf)
        treatment_df = disaster_joined.groupby(['grid_id', 'year']).size().reset_index(name='n_disasters')
        treatment_df['treated'] = 1
        first_treatment = treatment_df.groupby('grid_id')['year'].min().reset_index()
        first_treatment.columns = ['grid_id', 'first_treatment_year']
        treatment_df = treatment_df.merge(first_treatment, on='grid_id', how='left')
        print(f"✓ Treatment panel: {treatment_df['grid_id'].nunique():,} treated cells")
        treatment_df.to_csv(CONFIG['data_dir'] / 'processed' / 'treatment_panel.csv', index=False)
    else:
        treatment_df = None
else:
    print(f"⚠ EM-DAT file not found")
    treatment_df = None

# STEP 8: CREATE ANALYSIS PANEL
if occurrence_panel is not None and treatment_df is not None:
    panel_df = occurrence_panel.merge(treatment_df[['grid_id', 'year', 'treated', 'first_treatment_year']],
                                     on=['grid_id', 'year'], how='left')
    panel_df['treated'] = panel_df['treated'].fillna(0)
    panel_df = panel_df.merge(grid_gdf[['grid_id', 'lon_center', 'lat_center', 'area_km2']], on='grid_id', how='left')
    print(f"✓ Analysis panel: {len(panel_df):,} obs, {panel_df['grid_id'].nunique():,} cells, treatment rate: {panel_df['treated'].mean()*100:.2f}%")
    panel_df.to_csv(CONFIG['data_dir'] / 'processed' / 'analysis_panel.csv', index=False)
else:
    panel_df = None

# STEP 9: DOUBLE ML FUNCTIONS
def get_ml_model(model_type, task='regression'):
    if task == 'regression':
        if model_type == 'random_forest':
            return RandomForestRegressor(n_estimators=100, max_depth=10, min_samples_leaf=20, n_jobs=-1, random_state=42)
        elif model_type == 'gradient_boosting':
            return GradientBoostingRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42)
        else:
            return LassoCV(cv=5, random_state=42)
    else:
        return RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_leaf=20, n_jobs=-1, random_state=42)

def prepare_dml_features(panel_df):
    features = []
    feature_names = []
    if 'lon_center' in panel_df.columns:
        features.append(panel_df['lon_center'].values.reshape(-1, 1))
        feature_names.append('lon_center')
    if 'lat_center' in panel_df.columns:
        features.append(panel_df['lat_center'].values.reshape(-1, 1))
        feature_names.append('lat_center')
    if 'year' in panel_df.columns:
        features.append(panel_df['year'].values.reshape(-1, 1))
        feature_names.append('year')
    if 'area_km2' in panel_df.columns:
        features.append(panel_df['area_km2'].values.reshape(-1, 1))
        feature_names.append('area_km2')
    if 'species' in panel_df.columns:
        species_dummies = pd.get_dummies(panel_df['species'], prefix='sp')
        features.append(species_dummies.values)
        feature_names.extend(species_dummies.columns.tolist())
    if 'grid_id' in panel_df.columns:
        grid_hash = panel_df['grid_id'].apply(lambda x: hash(x) % 100)
        grid_dummies = pd.get_dummies(grid_hash, prefix='grid')
        features.append(grid_dummies.values)
        feature_names.extend(grid_dummies.columns.tolist())
    X = np.hstack(features)
    print(f"Feature matrix: {X.shape[0]:,} rows × {X.shape[1]} features")
    return X, feature_names

def double_ml_ate(Y, D, X, n_folds=5, ml_model='random_forest', seed=42):
    np.random.seed(seed)
    Y = np.array(Y)
    D = np.array(D)
    X = np.array(X) if not isinstance(X, np.ndarray) else X
    n = len(Y)
    Y_res = np.zeros(n)
    D_res = np.zeros(n)
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=seed)
    print(f"Running Double ML ({n_folds}-fold cross-fitting), n={n:,}, treatment rate={D.mean()*100:.2f}%")
    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(X)):
        print(f"  Fold {fold_idx + 1}/{n_folds}...", end=" ")
        X_train, X_test = X[train_idx], X[test_idx]
        Y_train, Y_test = Y[train_idx], Y[test_idx]
        D_train, D_test = D[train_idx], D[test_idx]
        y_model = get_ml_model(ml_model, task='regression')
        y_model.fit(X_train, Y_train)
        Y_pred = y_model.predict(X_test)
        Y_res[test_idx] = Y_test - Y_pred
        d_model = get_ml_model(ml_model, task='classification')
        d_model.fit(X_train, D_train)
        if hasattr(d_model, 'predict_proba'):
            D_pred = d_model.predict_proba(X_test)[:, 1]
        else:
            D_pred = d_model.predict(X_test)
        D_res[test_idx] = D_test - D_pred
        print("✓")
    numerator = np.mean(Y_res * D_res)
    denominator = np.mean(D_res ** 2)
    if denominator < 1e-10:
        print("⚠ Very small treatment variation")
        return None
    ate = numerator / denominator
    psi = (Y_res - ate * D_res) * D_res / denominator
    se = np.std(psi) / np.sqrt(n)
    ci_lower = ate - 1.96 * se
    ci_upper = ate + 1.96 * se
    pvalue = 2 * (1 - stats.norm.cdf(abs(ate / se)))
    print(f"\nDOUBLE ML RESULTS:")
    print(f"ATE: {ate:.6f}, SE: {se:.6f}, 95% CI: [{ci_lower:.6f}, {ci_upper:.6f}], p-value: {pvalue:.6f}")
    return {'ate': ate, 'se': se, 'ci_lower': ci_lower, 'ci_upper': ci_upper, 'pvalue': pvalue, 'residuals': (Y_res, D_res)}

def estimate_cate_by_group(Y, D, X, group_var, n_folds=5, ml_model='random_forest'):
    unique_groups = np.unique(group_var)
    results = {}
    print(f"\nEstimating CATE for {len(unique_groups)} groups...")
    for group in unique_groups:
        print(f"\n--- Group: {group} ---")
        mask = (group_var == group)
        if mask.sum() < 100:
            print(f"Skipping (n={mask.sum()})")
            continue
        try:
            result = double_ml_ate(Y[mask], D[mask], X[mask], n_folds=n_folds, ml_model=ml_model)
            if result is not None:
                results[group] = result
        except Exception as e:
            print(f"Error: {e}")
    return results

def double_ml_event_study(panel_df, outcome_var, entity_var='grid_id', time_var='year',
                          treatment_time_var='first_treatment_year', window=(-3, 5), ml_model='random_forest'):
    panel = panel_df.copy()
    panel['event_time'] = panel[time_var] - panel[treatment_time_var]
    X, feature_names = prepare_dml_features(panel)
    Y = panel[outcome_var].values
    results = []
    print(f"\nDouble ML Event Study: {window[0]} to {window[1]} years")
    for t in range(window[0], window[1] + 1):
        if t == -1:
            results.append({'event_time': t, 'ate': 0, 'se': 0, 'ci_lower': 0, 'ci_upper': 0, 'pvalue': 1.0})
            continue
        print(f"\nEvent time t = {t}...")
        D = (panel['event_time'] == t).astype(int).values
        if D.sum() < 50:
            print(f"  Skipping (n={D.sum()})")
            continue
        try:
            result = double_ml_ate(Y, D, X, ml_model=ml_model)
            if result is not None:
                results.append({'event_time': t, 'ate': result['ate'], 'se': result['se'],
                               'ci_lower': result['ci_lower'], 'ci_upper': result['ci_upper'], 'pvalue': result['pvalue']})
        except Exception as e:
            print(f"  Error: {e}")
    return pd.DataFrame(results)

def plot_dml_results(dml_result, title='Double ML Results', save_path=None):
    fig, ax = plt.subplots(figsize=(8, 6))
    ate = dml_result['ate']
    ci_lower = dml_result['ci_lower']
    ci_upper = dml_result['ci_upper']
    ax.scatter([0], [ate], s=200, color='darkblue', zorder=3)
    ax.errorbar([0], [ate], yerr=[[ate - ci_lower], [ci_upper - ate]],
                fmt='none', ecolor='darkblue', capsize=10, capthick=2, linewidth=2)
    ax.axhline(0, color='red', linestyle='--', linewidth=1, alpha=0.5)
    ax.set_xlim(-0.5, 0.5)
    ax.set_xticks([])
    ax.set_ylabel('Treatment Effect', fontsize=12)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.grid(alpha=0.3, axis='y')
    ax.text(0, ate + (ci_upper - ci_lower) * 0.5,
            f'ATE = {ate:.4f}\n95% CI: [{ci_lower:.4f}, {ci_upper:.4f}]\np = {dml_result["pvalue"]:.4f}',
            ha='center', va='bottom', fontsize=10, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

def plot_dml_event_study(event_study_df, save_path=None):
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(event_study_df['event_time'], event_study_df['ate'], 'o-', color='darkblue', linewidth=2, markersize=8)
    ax.fill_between(event_study_df['event_time'], event_study_df['ci_lower'], event_study_df['ci_upper'],
                    alpha=0.2, color='darkblue')
    ax.axhline(0, color='black', linestyle='-', linewidth=1)
    ax.axvline(-0.5, color='red', linestyle='--', linewidth=1.5, alpha=0.7)
    ax.set_xlabel('Years Relative to Disaster', fontsize=12)
    ax.set_ylabel('Treatment Effect', fontsize=12)
    ax.set_title('Double ML Event Study', fontsize=14, fontweight='bold')
    ax.grid(alpha=0.3)
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

def plot_cate_forest(cate_results, save_path=None):
    groups = list(cate_results.keys())
    ates = [r['ate'] for r in cate_results.values()]
    ci_lowers = [r['ci_lower'] for r in cate_results.values()]
    ci_uppers = [r['ci_upper'] for r in cate_results.values()]
    fig, ax = plt.subplots(figsize=(10, max(6, len(groups) * 0.4)))
    y_pos = range(len(groups))
    ax.errorbar(ates, y_pos, xerr=[[ate - ci_l for ate, ci_l in zip(ates, ci_lowers)],
                                     [ci_u - ate for ate, ci_u in zip(ates, ci_uppers)]],
                fmt='o', capsize=5, capthick=2, markersize=8, color='darkgreen')
    ax.axvline(0, color='red', linestyle='--', linewidth=1, alpha=0.5)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(groups)
    ax.set_xlabel('Treatment Effect (CATE)', fontsize=12)
    ax.set_title('Heterogeneous Effects by Group', fontsize=14, fontweight='bold')
    ax.grid(alpha=0.3, axis='x')
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

print("✓ Double ML functions defined")

# STEP 10: RUN DOUBLE ML ANALYSIS
if panel_df is not None:
    print("\n" + "="*80)
    print("RUNNING DOUBLE ML ANALYSIS")
    print("="*80)
    X, feature_names = prepare_dml_features(panel_df)
    Y = panel_df['occupancy'].values
    D = panel_df['treated'].values
    dml_main = double_ml_ate(Y, D, X, n_folds=CONFIG['n_folds'], ml_model=CONFIG['ml_model'])
    if dml_main is not None:
        pd.DataFrame([dml_main]).to_csv(CONFIG['results_dir'] / 'dml_main_results.csv', index=False)
        plot_dml_results(dml_main, title='Double ML: Disaster Impact on Species',
                        save_path=CONFIG['results_dir'] / 'dml_main_effect.png')
    event_study_df = double_ml_event_study(panel_df, outcome_var='occupancy', window=CONFIG['event_window'],
                                           ml_model=CONFIG['ml_model'])
    if len(event_study_df) > 0:
        event_study_df.to_csv(CONFIG['results_dir'] / 'dml_event_study.csv', index=False)
        plot_dml_event_study(event_study_df, save_path=CONFIG['results_dir'] / 'dml_event_study.png')
    if 'species' in panel_df.columns:
        cate_species = estimate_cate_by_group(Y, D, X, group_var=panel_df['species'].values, ml_model=CONFIG['ml_model'])
        if len(cate_species) > 0:
            cate_df = pd.DataFrame(cate_species).T
            cate_df.index.name = 'species'
            cate_df.to_csv(CONFIG['results_dir'] / 'dml_cate_species.csv')
            plot_cate_forest(cate_species, save_path=CONFIG['results_dir'] / 'dml_cate_species.png')
    print("\n✓ DOUBLE ML ANALYSIS COMPLETE!")
    print(f"Results saved to: {CONFIG['results_dir']}")
else:
    print("⚠ Cannot run analysis - panel_df not available")


Downloading...
From (original): https://drive.google.com/uc?id=19sq2HQipIoIcJvsID_M8iB84Ap-2lJje
From (redirected): https://drive.google.com/uc?id=19sq2HQipIoIcJvsID_M8iB84Ap-2lJje&confirm=t&uuid=b7a624b7-6eee-4898-9704-99acd506c7d7
To: /content/data.zip
100%|██████████| 9.01G/9.01G [01:13<00:00, 123MB/s]


✓ Data extracted
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.4/338.4 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 121.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 116.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 158.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 kB 25.0 MB/s eta 0:00:00
✓ Config set: endangered_asia_disaster_analysis_dml
✓ Data loading functions defined
✓ Species list: 13 species
✓ Grid created: 23,400 cells
Loaded 506,589 records
After filtering: 506,589 records
✓ Balanced panel: 7,605,000 rows, occupancy rate: 0.16%
Loaded 766 disaster events
✓ Treatment panel: 592 treated cells
✓ Analysis panel: 7,605,000 obs